In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
# Data processing
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
# Model and performance
import tensorflow as tf
from tensorflow.python.keras import layers, losses
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE_SEED = 12

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Kaggle/03-01-2018.csv')
df = df.drop(['Timestamp'], axis=1)
df.replace([-np.inf, np.inf], np.nan, inplace = True)
df.dropna(inplace=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328444 entries, 0 to 331124
Data columns (total 79 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Dst Port           328444 non-null  object
 1   Protocol           328444 non-null  object
 2   Flow Duration      328444 non-null  object
 3   Tot Fwd Pkts       328444 non-null  object
 4   Tot Bwd Pkts       328444 non-null  object
 5   TotLen Fwd Pkts    328444 non-null  object
 6   TotLen Bwd Pkts    328444 non-null  object
 7   Fwd Pkt Len Max    328444 non-null  object
 8   Fwd Pkt Len Min    328444 non-null  object
 9   Fwd Pkt Len Mean   328444 non-null  object
 10  Fwd Pkt Len Std    328444 non-null  object
 11  Bwd Pkt Len Max    328444 non-null  object
 12  Bwd Pkt Len Min    328444 non-null  object
 13  Bwd Pkt Len Mean   328444 non-null  object
 14  Bwd Pkt Len Std    328444 non-null  object
 15  Flow Byts/s        328444 non-null  object
 16  Flow Pkts/s        3

In [ ]:
df['Label'].value_counts()

Benign           236016
Infilteration     92403
Label                25
Name: Label, dtype: int64

In [ ]:
df = df[df['Label'] != 'Label']
df['Label'].value_counts()

Benign           236016
Infilteration     92403
Name: Label, dtype: int64

In [ ]:
df.replace(to_replace=["Infilteration"], value="Malicious", inplace=True)
df.shape

(328419, 79)

In [ ]:
df.replace(to_replace = 'Benign', value = 0, inplace=True)
df.replace(to_replace = 'Malicious', value = 1, inplace=True)

In [ ]:
df = df.astype(float)
df = df[np.isfinite(df).all(1)]

In [ ]:
# Initialize the scaler
scaler = MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), columns=df.columns)

In [ ]:
from google.colab import files
df.to_csv('dataset9.csv', encoding = 'utf-8-sig')
files.download('dataset9v.csv')

In [ ]:
dfmal = df.loc[df['Label'] == 1]
dfmal

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
116487,0.051713,0.352941,1.391001e-02,0.000383,0.000197,0.000115,0.000038,0.335874,0.0,0.096575,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
116488,0.051713,0.352941,1.388144e-02,0.000383,0.000197,0.000115,0.000038,0.335874,0.0,0.096575,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
116489,0.535225,0.352941,3.730250e-03,0.000000,0.000028,0.000000,0.000000,0.000000,0.0,0.000000,...,0.666667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
116490,0.000000,0.000000,9.386244e-01,0.003226,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.239474,0.466759,0.579825,0.043444,0.085,0.072296,0.1175,0.050913,1.0
116491,0.051713,0.352941,2.102523e-02,0.000547,0.000197,0.000116,0.000038,0.335874,0.0,0.070486,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291430,0.006790,0.352941,6.502917e-03,0.000219,0.000084,0.000005,0.000003,0.026931,0.0,0.007260,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
291431,0.051713,0.352941,2.447725e-02,0.000492,0.000197,0.000117,0.000038,0.344004,0.0,0.078630,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
291432,0.006790,0.352941,1.130108e-03,0.000109,0.000028,0.000000,0.000000,0.000000,0.0,0.000000,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0
291433,0.933989,0.352941,2.166667e-07,0.000055,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.416667,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,1.0


In [ ]:
dfben = df.loc[df['Label'] == 0]
dfben

,Dst Port,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0.000000,0.000000,0.960899,0.000219,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.015898,0.0,0.015898,0.015898,0.472500,0.108878,0.508333,0.437500,0.0
1,0.000000,0.000000,0.508312,0.000055,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.508333,0.000000,0.508333,0.508333,0.0
2,0.001022,1.000000,0.509575,0.000219,0.000000,0.000153,0.000000,0.152439,0.205479,0.205479,...,0.166667,0.030973,0.0,0.030973,0.030973,0.160000,0.226449,0.271667,0.066664,0.0
3,0.000000,0.000000,0.508313,0.000055,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.508333,0.000000,0.508333,0.508333,0.0
4,0.000000,0.000000,0.516646,0.000109,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.008771,0.0,0.008771,0.008771,0.508333,0.000000,0.508333,0.508333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328176,0.000000,0.000000,0.987451,0.001640,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
328177,0.040192,0.352941,0.000953,0.000055,0.000028,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
328178,0.051713,0.352941,0.011707,0.000383,0.000197,0.000116,0.000038,0.335874,0.000000,0.096918,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
328179,0.051713,0.352941,0.033056,0.000711,0.000197,0.000142,0.000009,0.345528,0.000000,0.067808,...,0.416667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [ ]:
df.loc[df['Label'] == 0].shape

(235778, 79)

In [ ]:
dfmal.shape

(92403, 79)

In [ ]:
df.loc[df['Label'] == 1].shape

(92403, 79)

In [ ]:
dfben.drop('Label', axis=1, inplace=True)
dfmal.drop('Label', axis=1, inplace=True)
dfben = dfben.values
dfmal = dfmal.values

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
train, test = train_test_split(dfben, test_size=0.25, random_state=42)

In [ ]:
train.shape

(176833, 78)

In [ ]:
from sklearn import metrics
import numpy as np
import pandas as pd
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout

model = Sequential()
model.add(Dense(78, input_dim=train.shape[1], activation='sigmoid'))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(8, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(train.shape[1]))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 78)                6162      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2528      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 16)                1

In [ ]:
model.fit(train, train, verbose=1, epochs=10)

Epoch 1/10
5527/5527 [==============================] - 19s 3ms/step - loss: 0.0227
Epoch 2/10
5527/5527 [==============================] - 15s 3ms/step - loss: 0.0196
Epoch 3/10
5527/5527 [==============================] - 15s 3ms/step - loss: 0.0094
Epoch 4/10
5527/5527 [==============================] - 14s 3ms/step - loss: 0.0057
Epoch 5/10
5527/5527 [==============================] - 14s 3ms/step - loss: 0.0044
Epoch 6/10
5527/5527 [==============================] - 15s 3ms/step - loss: 0.0038
Epoch 7/10
5527/5527 [==============================] - 14s 3ms/step - loss: 0.0036
Epoch 8/10
5527/5527 [==============================] - 16s 3ms/step - loss: 0.0034
Epoch 9/10
5527/5527 [==============================] - 15s 3ms/step - loss: 0.0033
Epoch 10/10
5527/5527 [==============================] - 16s 3ms/step - loss: 0.0031


In [ ]:
pred = model.predict(test)
score1 = np.sqrt(metrics.mean_squared_error(pred, test))

In [ ]:
pred

array([[ 3.18232447e-01,  2.69212127e-01,  9.14775252e-01, ...,
        -2.29658978e-03,  6.80513322e-01,  6.72453284e-01],
       [-9.14637093e-03,  9.95697498e-01,  1.61588266e-02, ...,
         6.38939999e-03,  1.89319253e-05, -1.30672455e-02],
       [ 6.81595951e-02,  3.42168987e-01,  3.15570533e-02, ...,
        -6.89597148e-03, -1.12916306e-02, -1.43003166e-02],
       ...,
       [ 4.95499164e-01,  4.08340991e-01,  2.77322605e-02, ...,
        -4.79146931e-03, -1.91872567e-03,  3.23782861e-03],
       [ 6.67600408e-02,  3.37410688e-01,  3.80022898e-02, ...,
        -6.65656012e-03, -9.58207995e-03, -1.24703348e-02],
       [ 2.63904065e-01,  3.50584567e-01,  3.35069001e-02, ...,
        -3.18115344e-03, -3.78040969e-03, -7.70895928e-03]], dtype=float32)

In [ ]:
pred = model.predict(test)
score2 = np.sqrt(metrics.mean_squared_error(pred, test))

In [ ]:
pred = model.predict(dfmal)
score3 = np.sqrt(metrics.mean_squared_error(pred, dfmal))

In [ ]:
print(f"Train Dataset Score (RMSE): {score1}".format(score1))
print(f"Test Dataset Score (RMSE): {score2}")
print(f"Malicious Dataset Score (RMSE): {score3}")

Train Dataset Score (RMSE): 0.049182698451536645
Test Dataset Score (RMSE): 0.049182698451536645
Malicious Dataset Score (RMSE): 0.05242198755934874
